In [1]:
# Install required packages in the folllowing few cells

In [2]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [3]:
%%capture
# hidden output
!pip install alpha_vantage

In [4]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [1]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume
import yfinance as yf
import time

from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


In [2]:
data_yf = pd.read_csv('data/all_stock_data.csv')

In [6]:
data_yf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
0,2000-09-23,31.6250,32.93750,29.31250,30.15625,18.661274,441758600.0,MSFT,-3.148325,-3.096951,...,-80.370370,-60.778443,-54.444444,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-1.774464e+09,-0.046443,0
1,2000-09-30,30.2500,30.40625,27.25000,27.78125,17.191576,423283200.0,MSFT,-3.424853,-3.162532,...,-95.527157,-80.370370,-60.778443,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-0.081612,0
2,2000-10-07,27.8125,28.46875,26.06250,26.87500,16.630777,416336400.0,MSFT,-3.674770,-3.264980,...,-89.714286,-95.527157,-80.370370,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-0.033708,0
3,2000-10-14,26.7500,33.06250,24.21875,32.59375,20.169643,728943800.0,MSFT,-3.372500,-3.286484,...,-93.041237,-89.714286,-95.527157,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,0.218458,1
4,2000-10-21,32.3125,34.59375,30.09375,33.84375,20.943165,686320600.0,MSFT,-2.997530,-3.228693,...,-96.255507,-93.041237,-89.714286,-3.552570e+09,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,0.047389,1


In [10]:
data_yf.isna().sum()


Date             0
Open             0
High             0
Low              0
Close            0
                ..
OBV_lag_3        0
OBV_lag_4        0
OBV_lag_5        0
gain_loss_pct    0
win              0
Length: 128, dtype: int64

In [309]:
data_yf.isna().any().any()

False

In [310]:
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,WILLR_14_lag_3,WILLR_14_lag_4,WILLR_14_lag_5,OBV_lag_1,OBV_lag_2,OBV_lag_3,OBV_lag_4,OBV_lag_5,gain_loss_pct,win
38,2000-09-23,31.625000,32.937500,29.312500,30.156250,19.109638,441758600.0,MSFT,-3.148325,-3.096951,...,-80.370370,-60.778443,-54.444444,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-1.774464e+09,-0.046443,0
39,2000-09-30,30.250000,30.406250,27.250000,27.781250,17.604628,423283200.0,MSFT,-3.424853,-3.162532,...,-95.527157,-80.370370,-60.778443,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-2.053990e+09,-0.081612,0
40,2000-10-07,27.812500,28.468750,26.062500,26.875000,17.030344,416336400.0,MSFT,-3.674770,-3.264980,...,-89.714286,-95.527157,-80.370370,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,-2.336127e+09,-0.033708,0
41,2000-10-14,26.750000,33.062500,24.218750,32.593750,20.654245,728943800.0,MSFT,-3.372500,-3.286484,...,-93.041237,-89.714286,-95.527157,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,-2.650566e+09,0.218458,1
42,2000-10-21,32.312500,34.593750,30.093750,33.843750,21.446360,686320600.0,MSFT,-2.997530,-3.228693,...,-96.255507,-93.041237,-89.714286,-3.552570e+09,-4.281514e+09,-3.865177e+09,-3.441894e+09,-3.000136e+09,0.047389,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562795,2024-03-04,27.469999,27.680000,26.600000,27.350000,27.350000,3164800.0,NWS,1.678421,1.526591,...,-19.534286,-22.121618,-9.001938,9.601709e+11,9.601741e+11,9.601718e+11,9.601682e+11,9.601627e+11,-0.004368,0
562796,2024-03-11,27.350000,27.910000,26.250000,26.440001,26.440001,5035500.0,NWS,1.576836,1.536640,...,-18.926556,-19.534286,-22.121618,9.601678e+11,9.601709e+11,9.601741e+11,9.601718e+11,9.601682e+11,-0.033272,0
562797,2024-03-18,26.570000,27.250000,26.250000,26.799999,26.799999,3302900.0,NWS,1.507996,1.530911,...,-22.652761,-18.926556,-19.534286,9.601627e+11,9.601678e+11,9.601709e+11,9.601741e+11,9.601718e+11,0.008656,1
562798,2024-03-25,26.799999,27.420000,26.790001,27.059999,27.059999,2564200.0,NWS,1.457616,1.516252,...,-23.397906,-22.652761,-18.926556,9.601661e+11,9.601627e+11,9.601678e+11,9.601709e+11,9.601741e+11,0.009702,1


In [23]:
first_appearance_dates = data_yf.groupby('Ticker')['Date'].min()
first_appearance_dates.head(20)

Ticker
A       2000-01-01
AAL     2005-09-26
AAPL    2000-01-01
ABBV    2013-01-01
ABNB    2020-12-07
ABT     2000-01-01
ACGL    2000-01-01
ACN     2001-07-16
ADBE    2000-01-01
ADI     2000-01-01
ADM     2000-01-01
ADP     2000-01-01
ADSK    2000-01-01
AEE     2000-01-01
AEP     2000-01-01
AES     2000-01-01
AFL     2000-01-01
AIG     2000-01-01
AIZ     2004-02-02
AJG     2000-01-01
Name: Date, dtype: object

In [11]:
# Initialize a list to collect DataFrames
#symbols = list(data_yf['Ticker'].value_counts().index)[0:300] #eases the number

symbols = list(data_yf['Ticker'].value_counts().index)


df_list = []

# Loop over each unique ticker
for ticker in range(len(symbols)):
    df = data_yf[data_yf['Ticker'] == symbols[ticker]]
    
    # Rename all columns in df to "original_column_nameTicker"
    df.columns = [str(col) + str(symbols[ticker]) for col in df.columns]
    
    df = df.reset_index().drop(columns=['index'])#[0:990]
    
    # Append modified df to the list
    df_list.append(df)



In [13]:
num_null = 0

for df in df_list:
    if df.isna().any().any() == True:
        num_null +=1 
        
data = pd.concat(df_list, axis=1)


data.set_index('DateFITB', inplace=True)

In [16]:
date_values = [col for col in data.columns if 'Date' in col]
data.drop(columns=date_values, inplace=True)

In [18]:
data.isna().sum()

OpenEXC                 0
HighEXC                 0
LowEXC                  0
CloseEXC                0
Adj CloseEXC            0
                     ... 
OBV_lag_3VLTO        1240
OBV_lag_4VLTO        1240
OBV_lag_5VLTO        1240
gain_loss_pctVLTO    1240
winVLTO              1240
Length: 60878, dtype: int64

In [25]:
data.shape

(1266, 60878)

Now, we will do Predict then Optimize -- Shared Learning

<h2> Predict then Optimize -- Shared Learning</h2>

In [418]:
from sklearn.preprocessing import StandardScaler

float_columns = data.select_dtypes(include=['float64']).columns

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize only the float columns
data[float_columns] = scaler.fit_transform(data[float_columns])

In [420]:
data = data[float_columns]

In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from scipy.optimize import minimize

In [ ]:
# Prepare features and targets for model training
def prepare_features_targets(data_2):
#     indicators = calculate_technical_indicators(data)
#     returns = np.log(data / data.shift(1)).add_suffix('_returns')
#     full_data = pd.concat([indicators, returns], axis=1).dropna()
    features = data_2[[col for col in data_2.columns if 'gain' not in col]]
    targets = data_2[[col for col in data_2.columns if 'gain' in col]]
    return features, targets

# Build and compile a neural network
def build_model(input_shape, output_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(output_shape)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Portfolio optimization function with variance consideration and risk aversion
def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
    weights = weights / np.sum(weights)  # Normalize weights
    pred_returns = model.predict(np.array([features.iloc[-1]]))[0]
    expected_return = np.dot(weights, pred_returns)
    covariance_matrix = historical_returns.cov()
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    # Objective function with risk aversion parameter
    return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance

# Main script


features, targets = prepare_features_targets(data)

ta

train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

model = build_model(train_features.shape[1], train_targets.shape[1])
model.fit(train_features, train_targets, epochs=3, batch_size=5)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
initial_weights = np.ones(len(symbols)) / len(symbols)  # Adjust this based on the actual number of features or stocks.
bounds = tuple((0, 1) for _ in symbols)




# optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.999999999999999999999999), bounds=bounds, constraints=constraints)
# optimized_weights = optimized_result.x


# Optimizing the portfolio on training data
train_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
optimized_weights_train = train_optimized_result.x

# Optimizing the portfolio on testing data
test_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
optimized_weights_test = test_optimized_result.x

# Calculating ROI
train_predicted_returns = model.predict(train_features)
train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

test_predicted_returns = model.predict(test_features)
test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment


In [466]:
print("Optimized Portfolio Weights (Train):", optimized_weights_train)
print("Training ROI:", train_roi)
print("Optimized Portfolio Weights (Test):", optimized_weights_test)
print("Testing ROI:", test_roi)

Optimized Portfolio Weights (Train): [1.42389068e-02 5.76666673e-03 1.15725697e-02 3.04017065e-16
 0.00000000e+00 4.23325660e-16 0.00000000e+00 0.00000000e+00
 1.55572166e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.83970070e-16 0.00000000e+00 2.17587352e-16 0.00000000e+00
 2.05298980e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.61159328e-18 0.00000000e+00 5.12875753e-03 2.09837689e-16
 6.57712613e-17 3.15632097e-03 5.91711806e-17 1.76236216e-02
 1.38327964e-02 0.00000000e+00 0.00000000e+00 8.85363835e-18
 0.00000000e+00 2.91313629e-16 1.36379743e-16 0.00000000e+00
 3.54754339e-17 0.00000000e+00 4.23394051e-18 0.00000000e+00
 4.76203617e-16 1.22907869e-16 2.40818243e-16 0.00000000e+00
 2.37499568e-17 4.02256278e-17 1.03969489e-16 1.49770672e-16
 0.00000000e+00 9.00836480e-17 1.65992200e-16 1.60748033e-16
 0.00000000e+00 0.00000000e+00 8.16260240e-17 1.13115202e-16
 1.65621199e-16 4.32558853e-02 0.00000000e+00 0.00000000e+00
 4.04365059e-17 5.61063088e-02 2.14547516e-16 0.

In [470]:
max(optimized_weights_train)

0.0801616812276641

In [469]:
train_predicted_returns

array([[ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986],
       [ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986],
       [ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986],
       ...,
       [ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986],
       [ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986],
       [ 0.02520421, -0.00050647,  0.00416604, ..., -0.01954383,
        -0.0222163 ,  0.01453986]], dtype=float32)

In [448]:
features, targets = prepare_features_targets(data_2)

In [449]:
targets

,gain_loss_pctEXC,gain_loss_pctFITB,gain_loss_pctDLTR,gain_loss_pctCAH,gain_loss_pctTROW,gain_loss_pctWEC,gain_loss_pctCHD,gain_loss_pctPNW,gain_loss_pctWST,gain_loss_pctHAS,...,gain_loss_pctSPG,gain_loss_pctNSC,gain_loss_pctKMB,gain_loss_pctROST,gain_loss_pctHES,gain_loss_pctCOF,gain_loss_pctJNJ,gain_loss_pctROP,gain_loss_pctHD,gain_loss_pctECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.694360,-0.951745,1.291177,5.119129,-1.376929,1.505437,0.130411,1.853300,-0.481889,-3.123023,...,1.342821,0.417655,0.776122,-4.028465,-0.103267,-0.909223,1.415533,-2.589745,-1.781906,0.632282
2000-01-08,1.386809,-0.274308,0.546186,-0.290533,2.201536,-1.391798,-0.475200,-0.983190,-0.264116,-0.131780,...,0.376781,0.604246,-0.662648,-2.209015,0.251704,0.415645,-1.312080,-0.497504,-0.068551,-0.600304
2000-01-15,0.470355,-1.312411,-2.058497,-3.083591,0.562699,0.814256,-2.702704,-0.043964,0.014470,-1.320626,...,0.616666,-1.445016,-0.415438,-0.317716,0.443612,-0.990464,-0.990539,1.879875,-0.519723,-2.018970
2000-01-22,1.810701,0.635857,-1.271614,-1.803783,-1.970153,-1.270763,-3.591030,-1.078796,0.460159,-0.138042,...,-0.367261,-3.046161,-0.721761,-1.015290,-2.342225,-0.523347,-2.584688,-1.363781,-2.072568,-2.601096
2000-01-29,1.280680,-0.102892,0.730785,0.481869,0.459804,0.946393,-3.065908,0.518188,-1.189605,0.590469,...,-0.309139,-1.986655,-1.001169,3.679701,-0.367906,-0.320923,-1.511668,1.079312,1.858605,0.022560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,-0.761131,-0.380816,-0.329512,-1.164647,-0.318284,-0.371444,0.296873,-0.040105,-1.115528,-0.488741,...,-0.435591,-0.994030,0.657039,-3.641276,-1.105799,-0.456450,-1.216989,-1.025464,-1.079533,-1.449416
2018-11-24,0.965086,0.446484,0.374207,0.584416,0.713270,1.044927,0.051992,0.337835,0.617311,-1.659189,...,0.309966,0.723517,0.892228,1.619663,-0.162410,0.473156,1.345199,0.737706,1.337693,1.242114
2018-12-01,0.371591,-1.307974,-1.021606,-0.523713,-1.610702,0.874492,0.140582,0.606426,-1.303425,-2.163113,...,0.187297,-2.141703,-0.684725,-2.700637,-1.294134,-1.444412,-0.092977,-1.450127,-1.377366,-1.909164


In [453]:
[col for col in targets.columns if ('EXC' or 'FITB') in col]

['gain_loss_pctEXC']

In [435]:
targets

,gain_loss_pctEXC,gain_loss_pctFITB,gain_loss_pctDLTR,gain_loss_pctCAH,gain_loss_pctTROW,gain_loss_pctWEC,gain_loss_pctCHD,gain_loss_pctPNW,gain_loss_pctWST,gain_loss_pctHAS,...,gain_loss_pctSPG,gain_loss_pctNSC,gain_loss_pctKMB,gain_loss_pctROST,gain_loss_pctHES,gain_loss_pctCOF,gain_loss_pctJNJ,gain_loss_pctROP,gain_loss_pctHD,gain_loss_pctECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.694360,-0.951745,1.291177,5.119129,-1.376929,1.505437,0.130411,1.853300,-0.481889,-3.123023,...,1.342821,0.417655,0.776122,-4.028465,-0.103267,-0.909223,1.415533,-2.589745,-1.781906,0.632282
2000-01-08,1.386809,-0.274308,0.546186,-0.290533,2.201536,-1.391798,-0.475200,-0.983190,-0.264116,-0.131780,...,0.376781,0.604246,-0.662648,-2.209015,0.251704,0.415645,-1.312080,-0.497504,-0.068551,-0.600304
2000-01-15,0.470355,-1.312411,-2.058497,-3.083591,0.562699,0.814256,-2.702704,-0.043964,0.014470,-1.320626,...,0.616666,-1.445016,-0.415438,-0.317716,0.443612,-0.990464,-0.990539,1.879875,-0.519723,-2.018970
2000-01-22,1.810701,0.635857,-1.271614,-1.803783,-1.970153,-1.270763,-3.591030,-1.078796,0.460159,-0.138042,...,-0.367261,-3.046161,-0.721761,-1.015290,-2.342225,-0.523347,-2.584688,-1.363781,-2.072568,-2.601096
2000-01-29,1.280680,-0.102892,0.730785,0.481869,0.459804,0.946393,-3.065908,0.518188,-1.189605,0.590469,...,-0.309139,-1.986655,-1.001169,3.679701,-0.367906,-0.320923,-1.511668,1.079312,1.858605,0.022560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,-0.761131,-0.380816,-0.329512,-1.164647,-0.318284,-0.371444,0.296873,-0.040105,-1.115528,-0.488741,...,-0.435591,-0.994030,0.657039,-3.641276,-1.105799,-0.456450,-1.216989,-1.025464,-1.079533,-1.449416
2018-11-24,0.965086,0.446484,0.374207,0.584416,0.713270,1.044927,0.051992,0.337835,0.617311,-1.659189,...,0.309966,0.723517,0.892228,1.619663,-0.162410,0.473156,1.345199,0.737706,1.337693,1.242114
2018-12-01,0.371591,-1.307974,-1.021606,-0.523713,-1.610702,0.874492,0.140582,0.606426,-1.303425,-2.163113,...,0.187297,-2.141703,-0.684725,-2.700637,-1.294134,-1.444412,-0.092977,-1.450127,-1.377366,-1.909164


In [433]:
train_features

,OpenEXC,HighEXC,LowEXC,CloseEXC,Adj CloseEXC,VolumeEXC,macdEXC,macd_signalEXC,rsiEXC,bb_highEXC,...,WILLR_14_lag_1ECL,WILLR_14_lag_2ECL,WILLR_14_lag_3ECL,WILLR_14_lag_4ECL,WILLR_14_lag_5ECL,OBV_lag_1ECL,OBV_lag_2ECL,OBV_lag_3ECL,OBV_lag_4ECL,OBV_lag_5ECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2014-06-07,-0.242783,-0.289041,-0.350776,-0.349225,0.329049,5.371132,0.414958,0.407902,0.846357,-0.268272,...,0.810639,0.407003,0.025839,-0.435271,-0.559021,1.050010,1.031846,1.013188,0.990402,0.954931
2004-02-14,-0.529281,-0.543852,-0.508593,-0.551057,-1.067444,-1.051845,0.292564,0.320210,0.871641,-0.390139,...,0.282450,0.937585,0.303877,-0.643824,0.212503,-0.662899,-0.622111,-0.658451,-0.703021,-0.667828
2005-08-20,0.752132,0.745367,0.843574,0.781306,0.014098,-0.956470,0.461429,0.499823,1.096661,0.093472,...,-0.968223,0.104545,0.272183,0.756654,-0.702422,-0.520629,-0.502473,-0.479000,-0.447543,-0.494904
2000-12-09,-0.532683,-0.506932,-0.574672,-0.611742,-1.316715,-0.749189,-0.105982,-0.454163,-0.250897,-0.375440,...,1.209985,1.203545,1.208542,1.030145,0.865993,-0.871274,-0.918496,-0.961546,-0.978490,-1.000888
2013-10-19,-0.819181,-0.849427,-0.859024,-0.878351,-0.397171,3.023186,0.041613,0.063883,-1.280976,-0.425300,...,1.323617,1.300793,1.323393,1.077827,1.080499,0.922866,0.897013,0.859858,0.830249,0.861769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-01-12,-1.169987,-1.144454,-1.156381,-1.188940,-1.608641,-0.893264,-0.011134,-0.091462,-0.769630,-0.547718,...,0.781089,1.242183,1.318641,1.203773,0.434557,-1.095812,-1.042420,-1.069356,-1.086596,-1.116615
2005-03-05,0.401666,0.342548,0.397890,0.359232,-0.343522,-0.704178,0.474462,0.496720,1.376079,-0.082163,...,-0.787521,-2.234757,-2.294933,-1.478303,-1.687343,-0.410815,-0.457547,-0.415225,-0.364485,-0.311810
2016-06-25,-0.446258,-0.354309,-0.412637,-0.302176,0.716848,1.146873,0.322969,0.347275,1.161279,-0.317593,...,-0.126950,0.745054,1.074175,1.225748,1.168850,1.573389,1.616175,1.669028,1.629716,1.601852


In [434]:
train_targets

,gain_loss_pctEXC,gain_loss_pctFITB,gain_loss_pctDLTR,gain_loss_pctCAH,gain_loss_pctTROW,gain_loss_pctWEC,gain_loss_pctCHD,gain_loss_pctPNW,gain_loss_pctWST,gain_loss_pctHAS,...,gain_loss_pctSPG,gain_loss_pctNSC,gain_loss_pctKMB,gain_loss_pctROST,gain_loss_pctHES,gain_loss_pctCOF,gain_loss_pctJNJ,gain_loss_pctROP,gain_loss_pctHD,gain_loss_pctECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2014-06-07,-1.321242,0.037264,-1.094734,-1.077156,-0.701242,-0.689975,-0.169797,-0.521178,-0.919824,-0.137430,...,-0.798851,-0.191413,-0.354067,-0.770036,0.497763,-0.146596,-0.266413,0.173555,-0.783489,-0.776387
2004-02-14,-0.309844,-0.172247,-0.167289,-0.271061,-0.607075,-0.920727,-0.001225,0.374647,0.289469,-0.270278,...,-0.214624,-0.110264,0.974072,0.143198,0.242885,-0.557088,-0.932211,-1.350096,-0.404274,-0.000536
2005-08-20,0.214239,-0.540859,-0.040589,-0.428474,0.067205,0.417981,-0.417263,0.216309,0.662870,-0.312831,...,-0.201437,-0.217559,-0.367498,-0.180147,-0.375097,-0.764113,-1.359104,-0.255055,-0.435678,-0.629637
2000-12-09,-1.101023,-0.584064,-4.000801,0.083265,-1.094890,-2.996681,-3.276181,-0.921375,-0.512728,-0.862049,...,0.254808,-1.898353,-0.612034,-2.100970,-0.656654,-0.493701,1.028348,-0.713703,-2.070134,-2.136503
2013-10-19,-0.932863,0.190959,0.173208,-0.549259,-0.416088,0.968277,0.763835,0.866286,0.200341,1.612387,...,0.220590,2.090388,2.561231,0.749904,-0.246310,-0.356849,0.149579,0.067117,0.469272,0.619503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-01-12,-0.339567,0.332021,0.986544,0.036110,-0.749127,0.001244,1.508460,0.540967,-1.048090,-0.943120,...,-0.014691,0.275257,0.693022,-0.492712,-0.414021,0.754461,0.650838,-0.816798,-0.948438,-0.015617
2005-03-05,-0.449429,-0.566823,-0.413274,-0.476754,-0.679812,-0.916729,-0.585727,-0.450252,-1.204553,-0.083896,...,-1.137957,0.302821,-0.343925,-0.816055,-1.233303,0.567430,-0.233129,-0.176062,-0.388050,-0.713884
2016-06-25,1.902652,0.162679,0.450846,1.111187,1.081853,0.847293,1.424114,1.324504,1.468691,0.602782,...,1.328553,0.902082,1.776158,0.759710,1.886711,0.510958,2.163801,0.562416,0.749023,1.025042


In [398]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [421]:
column_values = [col for col in data.columns if 'EXC' or 'APPL' in col]
data_2 = data[column_values]

['OpenEXC',
 'HighEXC',
 'LowEXC',
 'CloseEXC',
 'Adj CloseEXC',
 'VolumeEXC',
 'macdEXC',
 'macd_signalEXC',
 'rsiEXC',
 'bb_highEXC',
 'adxEXC',
 'atrEXC',
 'cciEXC',
 'mfiEXC',
 'soEXC',
 'kcEXC',
 'vwapEXC',
 'eomEXC',
 'vptEXC',
 'RSI_14EXC',
 'MFI_14EXC',
 'ROC_10EXC',
 'WILLR_14EXC',
 'OBVEXC',
 'macd_lag_1EXC',
 'macd_lag_2EXC',
 'macd_lag_3EXC',
 'macd_lag_4EXC',
 'macd_lag_5EXC',
 'macd_signal_lag_1EXC',
 'macd_signal_lag_2EXC',
 'macd_signal_lag_3EXC',
 'macd_signal_lag_4EXC',
 'macd_signal_lag_5EXC',
 'rsi_lag_1EXC',
 'rsi_lag_2EXC',
 'rsi_lag_3EXC',
 'rsi_lag_4EXC',
 'rsi_lag_5EXC',
 'bb_high_lag_1EXC',
 'bb_high_lag_2EXC',
 'bb_high_lag_3EXC',
 'bb_high_lag_4EXC',
 'bb_high_lag_5EXC',
 'adx_lag_1EXC',
 'adx_lag_2EXC',
 'adx_lag_3EXC',
 'adx_lag_4EXC',
 'adx_lag_5EXC',
 'atr_lag_1EXC',
 'atr_lag_2EXC',
 'atr_lag_3EXC',
 'atr_lag_4EXC',
 'atr_lag_5EXC',
 'cci_lag_1EXC',
 'cci_lag_2EXC',
 'cci_lag_3EXC',
 'cci_lag_4EXC',
 'cci_lag_5EXC',
 'mfi_lag_1EXC',
 'mfi_lag_2EXC',
 'm

In [429]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Prepare features and targets for model training
def prepare_features_targets(data):
    features = data[[col for col in data.columns if 'gain' not in col]]
    targets = data[[col for col in data.columns if 'gain' in col]]
    return features, targets

# Build and compile a neural network
def build_model(input_shape, output_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(output_shape)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Portfolio optimization function with variance consideration and risk aversion
def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
    weights = weights / np.sum(weights)  # Normalize weights
    pred_returns = model.predict(features)[-1]  # Predicting the last set of features
    expected_return = np.dot(weights, pred_returns)
    covariance_matrix = historical_returns.cov()
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance

column_values = [col for col in data.columns if 'EXC' or 'APPL' in col]
data_2 = data[column_values]

symbols = ['EXC', 'AAPL']

# Prepare features and targets
features, targets = prepare_features_targets(data_2)

# Split data into training and testing sets
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# Build model
model = build_model(train_features.shape[1], train_targets.shape[1])

# Train model
model.fit(train_features, train_targets, epochs=3, batch_size=32)

# Optimizing portfolio using training data
# train_returns = train_features.pct_change().dropna()
initial_weights = np.ones(len(symbols)) / len(symbols)
bounds = tuple((0, 1) for _ in symbols)
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

optimized_result_train = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
optimized_weights_train = optimized_result_train.x

# Calculate ROI for the training set
train_predicted_returns = model.predict(train_features)
train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# Optimizing portfolio using testing data
test_returns = test_features.pct_change().dropna()
optimized_result_test = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
optimized_weights_test = optimized_result_test.x

# Calculate ROI for the testing set
test_predicted_returns = model.predict(test_features)
test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

print(f"Training ROI: ${train_roi:.2f}")
print(f"Testing ROI: ${test_roi:.2f}")
print("Optimized Weights for Training Set:", optimized_weights_train)
print("Optimized Weights for Testing Set:", optimized_weights_test)


Epoch 1/3
25/25 [==============================] - 1s 28ms/step - loss: 2.0027
Epoch 2/3
25/25 [==============================] - 1s 27ms/step - loss: 1.0523
Epoch 3/3
25/25 [==============================] - 0s 5ms/step


ValueError: shapes (2,) and (300,) not aligned: 2 (dim 0) != 300 (dim 0)

In [426]:
features

,OpenEXC,HighEXC,LowEXC,CloseEXC,Adj CloseEXC,VolumeEXC,macdEXC,macd_signalEXC,rsiEXC,bb_highEXC,...,WILLR_14_lag_1ECL,WILLR_14_lag_2ECL,WILLR_14_lag_3ECL,WILLR_14_lag_4ECL,WILLR_14_lag_5ECL,OBV_lag_1ECL,OBV_lag_2ECL,OBV_lag_3ECL,OBV_lag_4ECL,OBV_lag_5ECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,-1.604497,-1.526012,-1.637917,-1.580927,-1.905666,-1.310734,-0.297377,1.394268,-2.605240,6.497919,...,1.357271,1.246743,1.206968,1.183555,1.235757,-0.977408,-0.998689,-1.096580,-1.254171,-1.441877
2000-01-08,-1.525812,-1.472445,-1.512700,-1.472255,-1.845120,-1.385226,-2.407280,0.594932,-2.573027,7.022594,...,-2.462696,1.355034,1.244626,1.207302,1.181531,-1.008601,-0.976169,-0.997250,-1.095038,-1.252316
2000-01-15,-1.476900,-1.470385,-1.488536,-1.459470,-1.837997,-1.302062,-4.028899,-0.394346,-2.568968,7.357843,...,-2.467414,-2.461944,1.352838,1.244966,1.205259,-1.033409,-1.007428,-0.974686,-0.995523,-1.092947
2000-01-22,-1.455634,-1.357072,-1.435813,-1.380630,-1.794072,-0.224300,-5.239015,-1.446810,-2.542141,7.563477,...,-2.497296,-2.466658,-2.461328,1.353195,1.242895,-1.055147,-1.032289,-1.006006,-0.972917,-0.993283
2000-01-29,-1.374823,-1.317927,-1.372106,-1.318836,-1.759645,-0.638543,-6.115640,-2.477885,-2.519668,7.674148,...,-2.501296,-2.496517,-2.466039,-2.461604,1.351041,-1.079659,-1.054074,-1.030914,-1.004295,-0.970643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,0.346544,0.295739,0.332513,0.273317,2.083922,-0.257580,0.311863,0.328352,0.747883,-0.115083,...,1.178573,1.171371,0.452393,-1.014532,-0.784028,2.256275,2.228335,2.195641,2.117683,2.188586
2018-11-24,0.279853,0.304310,0.341652,0.378324,2.243942,0.213902,0.325848,0.331781,1.224792,-0.104938,...,0.130422,1.176477,1.169310,0.452601,-1.014877,2.216881,2.264389,2.236050,2.203331,2.124572
2018-12-01,0.375806,0.366941,0.437256,0.417190,2.303171,0.369214,0.341229,0.337841,1.377717,-0.094248,...,1.319219,0.129145,1.174412,1.169637,0.451135,2.275573,2.224911,2.272174,2.243816,2.210347


In [427]:
targets

,gain_loss_pctEXC,gain_loss_pctFITB,gain_loss_pctDLTR,gain_loss_pctCAH,gain_loss_pctTROW,gain_loss_pctWEC,gain_loss_pctCHD,gain_loss_pctPNW,gain_loss_pctWST,gain_loss_pctHAS,...,gain_loss_pctSPG,gain_loss_pctNSC,gain_loss_pctKMB,gain_loss_pctROST,gain_loss_pctHES,gain_loss_pctCOF,gain_loss_pctJNJ,gain_loss_pctROP,gain_loss_pctHD,gain_loss_pctECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.694360,-0.951745,1.291177,5.119129,-1.376929,1.505437,0.130411,1.853300,-0.481889,-3.123023,...,1.342821,0.417655,0.776122,-4.028465,-0.103267,-0.909223,1.415533,-2.589745,-1.781906,0.632282
2000-01-08,1.386809,-0.274308,0.546186,-0.290533,2.201536,-1.391798,-0.475200,-0.983190,-0.264116,-0.131780,...,0.376781,0.604246,-0.662648,-2.209015,0.251704,0.415645,-1.312080,-0.497504,-0.068551,-0.600304
2000-01-15,0.470355,-1.312411,-2.058497,-3.083591,0.562699,0.814256,-2.702704,-0.043964,0.014470,-1.320626,...,0.616666,-1.445016,-0.415438,-0.317716,0.443612,-0.990464,-0.990539,1.879875,-0.519723,-2.018970
2000-01-22,1.810701,0.635857,-1.271614,-1.803783,-1.970153,-1.270763,-3.591030,-1.078796,0.460159,-0.138042,...,-0.367261,-3.046161,-0.721761,-1.015290,-2.342225,-0.523347,-2.584688,-1.363781,-2.072568,-2.601096
2000-01-29,1.280680,-0.102892,0.730785,0.481869,0.459804,0.946393,-3.065908,0.518188,-1.189605,0.590469,...,-0.309139,-1.986655,-1.001169,3.679701,-0.367906,-0.320923,-1.511668,1.079312,1.858605,0.022560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,-0.761131,-0.380816,-0.329512,-1.164647,-0.318284,-0.371444,0.296873,-0.040105,-1.115528,-0.488741,...,-0.435591,-0.994030,0.657039,-3.641276,-1.105799,-0.456450,-1.216989,-1.025464,-1.079533,-1.449416
2018-11-24,0.965086,0.446484,0.374207,0.584416,0.713270,1.044927,0.051992,0.337835,0.617311,-1.659189,...,0.309966,0.723517,0.892228,1.619663,-0.162410,0.473156,1.345199,0.737706,1.337693,1.242114
2018-12-01,0.371591,-1.307974,-1.021606,-0.523713,-1.610702,0.874492,0.140582,0.606426,-1.303425,-2.163113,...,0.187297,-2.141703,-0.684725,-2.700637,-1.294134,-1.444412,-0.092977,-1.450127,-1.377366,-1.909164


In [346]:
targets

,gain_loss_pctEXC,gain_loss_pctFITB,gain_loss_pctDLTR,gain_loss_pctCAH,gain_loss_pctTROW,gain_loss_pctWEC,gain_loss_pctCHD,gain_loss_pctPNW,gain_loss_pctWST,gain_loss_pctHAS,...,gain_loss_pctSPG,gain_loss_pctNSC,gain_loss_pctKMB,gain_loss_pctROST,gain_loss_pctHES,gain_loss_pctCOF,gain_loss_pctJNJ,gain_loss_pctROP,gain_loss_pctHD,gain_loss_pctECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,0.694360,-0.951745,1.291177,5.119129,-1.376929,1.505437,0.130411,1.853300,-0.481889,-3.123023,...,1.342821,0.417655,0.776122,-4.028465,-0.103267,-0.909223,1.415533,-2.589745,-1.781906,0.632282
2000-01-08,1.386809,-0.274308,0.546186,-0.290533,2.201536,-1.391798,-0.475200,-0.983190,-0.264116,-0.131780,...,0.376781,0.604246,-0.662648,-2.209015,0.251704,0.415645,-1.312080,-0.497504,-0.068551,-0.600304
2000-01-15,0.470355,-1.312411,-2.058497,-3.083591,0.562699,0.814256,-2.702704,-0.043964,0.014470,-1.320626,...,0.616666,-1.445016,-0.415438,-0.317716,0.443612,-0.990464,-0.990539,1.879875,-0.519723,-2.018970
2000-01-22,1.810701,0.635857,-1.271614,-1.803783,-1.970153,-1.270763,-3.591030,-1.078796,0.460159,-0.138042,...,-0.367261,-3.046161,-0.721761,-1.015290,-2.342225,-0.523347,-2.584688,-1.363781,-2.072568,-2.601096
2000-01-29,1.280680,-0.102892,0.730785,0.481869,0.459804,0.946393,-3.065908,0.518188,-1.189605,0.590469,...,-0.309139,-1.986655,-1.001169,3.679701,-0.367906,-0.320923,-1.511668,1.079312,1.858605,0.022560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,-0.761131,-0.380816,-0.329512,-1.164647,-0.318284,-0.371444,0.296873,-0.040105,-1.115528,-0.488741,...,-0.435591,-0.994030,0.657039,-3.641276,-1.105799,-0.456450,-1.216989,-1.025464,-1.079533,-1.449416
2018-11-24,0.965086,0.446484,0.374207,0.584416,0.713270,1.044927,0.051992,0.337835,0.617311,-1.659189,...,0.309966,0.723517,0.892228,1.619663,-0.162410,0.473156,1.345199,0.737706,1.337693,1.242114
2018-12-01,0.371591,-1.307974,-1.021606,-0.523713,-1.610702,0.874492,0.140582,0.606426,-1.303425,-2.163113,...,0.187297,-2.141703,-0.684725,-2.700637,-1.294134,-1.444412,-0.092977,-1.450127,-1.377366,-1.909164


In [348]:
features

,OpenEXC,HighEXC,LowEXC,CloseEXC,Adj CloseEXC,VolumeEXC,TickerEXC,macdEXC,macd_signalEXC,rsiEXC,...,WILLR_14_lag_2ECL,WILLR_14_lag_3ECL,WILLR_14_lag_4ECL,WILLR_14_lag_5ECL,OBV_lag_1ECL,OBV_lag_2ECL,OBV_lag_3ECL,OBV_lag_4ECL,OBV_lag_5ECL,winECL
DateFITB,,,,,,,,,,,,,,,,,,,,,
2000-01-01,-1.604497,-1.526012,-1.637917,-1.580927,-1.905666,-1.310734,EXC,-0.297377,1.394268,-2.605240,...,1.246743,1.206968,1.183555,1.235757,-0.977408,-0.998689,-1.096580,-1.254171,-1.441877,1
2000-01-08,-1.525812,-1.472445,-1.512700,-1.472255,-1.845120,-1.385226,EXC,-2.407280,0.594932,-2.573027,...,1.355034,1.244626,1.207302,1.181531,-1.008601,-0.976169,-0.997250,-1.095038,-1.252316,0
2000-01-15,-1.476900,-1.470385,-1.488536,-1.459470,-1.837997,-1.302062,EXC,-4.028899,-0.394346,-2.568968,...,-2.461944,1.352838,1.244966,1.205259,-1.033409,-1.007428,-0.974686,-0.995523,-1.092947,0
2000-01-22,-1.455634,-1.357072,-1.435813,-1.380630,-1.794072,-0.224300,EXC,-5.239015,-1.446810,-2.542141,...,-2.466658,-2.461328,1.353195,1.242895,-1.055147,-1.032289,-1.006006,-0.972917,-0.993283,0
2000-01-29,-1.374823,-1.317927,-1.372106,-1.318836,-1.759645,-0.638543,EXC,-6.115640,-2.477885,-2.519668,...,-2.496517,-2.466039,-2.461604,1.351041,-1.079659,-1.054074,-1.030914,-1.004295,-0.970643,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-17,0.346544,0.295739,0.332513,0.273317,2.083922,-0.257580,EXC,0.311863,0.328352,0.747883,...,1.171371,0.452393,-1.014532,-0.784028,2.256275,2.228335,2.195641,2.117683,2.188586,0
2018-11-24,0.279853,0.304310,0.341652,0.378324,2.243942,0.213902,EXC,0.325848,0.331781,1.224792,...,1.176477,1.169310,0.452601,-1.014877,2.216881,2.264389,2.236050,2.203331,2.124572,1
2018-12-01,0.375806,0.366941,0.437256,0.417190,2.303171,0.369214,EXC,0.341229,0.337841,1.377717,...,0.129145,1.174412,1.169637,0.451135,2.275573,2.224911,2.272174,2.243816,2.210347,0


Non Shared Learning


Next steps:

1. create functions to request a list of 100 stocks and get comprehensive data from alpha vantage
2. compare the data sources between AV and YF and decide what info to merge
3. decide how to incorporate the financials from each stock (meaning we get quarterly data for example for APPLE in the dataframe: `apple_quarterly_reports` ) this data has dimensions: 58x29 , therefore not sure how to incorporate it whith dataframe 15306 x 5


In [ ]:
data_yf

In [ ]:
#Need a train loader and a test loader
#Need to convert to optDataset, then wrap with DataLoader

In [ ]:
!pip install PyPl gurobipy pyepo

Create a LSTM-based model for stock prediction tasks

In [ ]:
import torch.nn.functional as F
from torch import nn
import torch 
class StockLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(StockLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        all_h, (h, c) = self.lstm(x)
        out = self.fc(all_h) # Apply Linear layer to outputs from all the hidden state.
        return out

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader

m = 50 # change based on number of assets
cov = np.cov(np.random.randn(10, m), rowvar=False) # covariance matrix
optmodel = pyepo.model.grb.portfolioModel(m, cov) # build model

Auto-Sklearn cannot be imported.
Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    train_loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        train_loss = 0
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            elif method_name == "mse":
                loss = loss_func(cp, c)
            else:
                raise ValueError("Method name {} not supported".format(method_name))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            train_loss += loss.item()
        train_loss /= len(loader_train)
        train_loss_log.append(train_loss)
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, train_loss, regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return train_loss_log, loss_log_regret

Create a Predict-then-Optimize Model

In [ ]:

spop = pyepo.func.SPOPlus(optmodel, processes=1)

In [ ]:
# Hyperparameters
VOCAB_SIZE = #fill in once data is imported
EMBEDDING_DIM = #param we can optimize
HIDDEN_DIM = #also a param to optimize
learning_rate = #another optimizable param
epoch = #optimize
num_layers = #optimize
epochs = 20
learning_rate = 2e-3
method_name = "spo+"

# Instantiate the model
lstm = StockLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,num_layers=num_layers)
loss_log_lstm_spo, loss_log_regret_lstm_spo = trainModel(lstm, loss_func=spop, method_name=method_name)
